<a href="https://colab.research.google.com/github/CThanhNg03/2-stupid-guys/blob/main/DmeHung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Fairseq & Requirements

In [ ]:
!pip install fairseq
!pip install sacremoses
!pip install sacrebleu
!pip install fastbpe
!pip install sentencepiece
!pip install TensorRT
!pip install tensorboardX
!pip install subword-nmt
!pip install transformers
!pip install evaluate
!pip install accelerate -U
!pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 27.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11291805 sha256=d5d0ec99ae4e182d8f6f331159924ce039bd936600f269cb5de5ffc671884833
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl

# Importing other packages

In [ ]:
import torch
import logging
import pandas as pd
import random
import numpy as np
import evaluate
import accelerate
from datasets import load_dataset
from fairseq.models.transformer import TransformerModel
from torch.utils.data import DataLoader
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments, Seq2SeqTrainer,
                          pipeline)
from sklearn.model_selection import train_test_split

In [ ]:
# Manual settings
logging.getLogger().setLevel(logging.WARNING)

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# ALT + OPUS Dataset in action

## Preprocess Raw Data from ALT Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/ColabNotebooks/data/Train/train2023.vi', 'r', encoding='utf-8') as file:
  lines = file.readlines()

processed_alt_vi = [line.strip() for line in lines]

with open('/content/drive/MyDrive/ColabNotebooks/data/Train/train2023.vi', 'w', encoding='utf-8') as file:
  file.write('\n'.join(processed_alt_vi))

In [ ]:
with open('/content/drive/MyDrive/ColabNotebooks/data/Train/train2023.lo', 'r', encoding='utf-8') as file:
  lines = file.readlines()

processed_alt_lo = [line.strip() for line in lines]

with open('/content/drive/MyDrive/ColabNotebooks/data/Train/train2023.lo', 'w', encoding='utf-8') as file:
  file.write('\n'.join(processed_alt_lo))

## Preprocess Raw Data from OPUS Dataset

In [ ]:
# with open('/kaggle/input/lo-vi-machine-translation/Data/Parallel/MultiCCAligned.lo-vi.lo', 'r', encoding='utf-8') as file:
#   opus_lo_lines = file.readlines()

# with open('/kaggle/working/processed_opus.lo_vi.lo', 'w', encoding='utf-8') as file:
#   file.write('\n'.join(opus_lo_lines))

In [ ]:
# with open('/kaggle/input/lo-vi-machine-translation/Data/Parallel/MultiCCAligned.lo-vi.vi', 'r', encoding='utf-8') as file:
#   opus_vi_lines = file.readlines()

# with open('/kaggle/working/processed_opus.lo_vi.vi', 'w', encoding='utf-8') as file:
#   file.write('\n'.join(opus_vi_lines))

In [ ]:
# !wc -l processed_opus.lo_vi.lo
# !wc -l processed_opus.lo_vi.vi

In [ ]:
# !head -n 5 processed_opus.lo_vi.lo
# !head -n 5 processed_opus.lo_vi.vi

# DataFrame Storage

In [ ]:
with open('/content/drive/MyDrive/ColabNotebooks/data/Train/train2023.vi', 'r', encoding='utf-8') as file:
  vie_data = file.readlines()
vie_df = pd.DataFrame(vie_data, columns = ['vie_Latn'])

# with open('processed_opus.lo_vi.vi', 'r', encoding='utf-8') as file:
#   vie_data = file.readlines()
# vie_df = pd.concat([vie_df, pd.DataFrame(vie_data, columns = ['vie_Latn'])], ignore_index=True)

In [ ]:
with open('/content/drive/MyDrive/ColabNotebooks/data/Train/train2023.lo', 'r', encoding='utf-8') as file:
  lao_data = file.readlines()
lao_df = pd.DataFrame(lao_data, columns = ['lao_Laoo'])

# with open('processed_opus.lo_vi.lo', 'r', encoding='utf-8') as file:
#   lao_data = file.readlines()
# lao_df = pd.concat([lao_df, pd.DataFrame(lao_data, columns = ['lao_Laoo'])], ignore_index=True)

In [ ]:
df = pd.concat([lao_df, vie_df], axis = 1)
df

,lao_Laoo,vie_Latn
0,ຖ້າ ເຈົ້າ ຮູ້ສຶກ ຢ້ານ ທີ່ ຈະ ປະກາດ ຂໍ ໃຫ້ ຊ້ອມ...,Nếu anh chị cảm thấy e ngại khi giới thiệu về ...
1,ຄົນ ຕໍ່າ ຕ້ອຍ ຈໍານວນ ຫຼາຍ ຄົງ ຈະ ຫຼົບ ຫຼີກ ຜູ້...,Hẳn nhiều người thấp kém e sợ và tránh mặt nhữ...
2,21 ບັດ ນີ້ເມື່ອລາຊິນີ ເຫັນ ຄວາມ ຢ້ານ ກົວ ຂອງ ພ...,"21 Bấy giờ, khi hoàng hậu trông thấy vẻ kinh h..."
3,ການ ຕັດສິນ ໃຈ ທີ່ ຈະ ປ່ຽນແປງ ກໍ ເປັນ ຂອງ ທ່ານ ...,Quyết định để thay đổi thuộc vào chính các anh...
4,ການ ເລືອກ ທີ່ ພວກ ເຈົ້າ ເຮັດ ໃນ ເວລາ ນີ້ ມີ ຄວ...,Những lựa chọn của các em ngay trước mắt là qu...
...,...,...
99995,ປະຕູສັງກະສີຈັບ-Z1961\n,cửa Kẽm xử lý-Z1961\n
99996,ກັ່ນຕອງ MS-5130\n,Lọc MS-5130\n
99997,cushion ບ່ອນນັ່ງ mold 6cm\n,Gối ghế khuôn 6cm\n
99998,Scalpers ຍິນດີຕ້ອນຮັບ\n,Chào mừng bạn\n


In [ ]:
df = df[~df['lao_Laoo'].str.startswith("\n")]

In [ ]:
df.sample(frac=1)
df.reset_index(drop=True)

,lao_Laoo,vie_Latn
0,ຖ້າ ເຈົ້າ ຮູ້ສຶກ ຢ້ານ ທີ່ ຈະ ປະກາດ ຂໍ ໃຫ້ ຊ້ອມ...,Nếu anh chị cảm thấy e ngại khi giới thiệu về ...
1,ຄົນ ຕໍ່າ ຕ້ອຍ ຈໍານວນ ຫຼາຍ ຄົງ ຈະ ຫຼົບ ຫຼີກ ຜູ້...,Hẳn nhiều người thấp kém e sợ và tránh mặt nhữ...
2,21 ບັດ ນີ້ເມື່ອລາຊິນີ ເຫັນ ຄວາມ ຢ້ານ ກົວ ຂອງ ພ...,"21 Bấy giờ, khi hoàng hậu trông thấy vẻ kinh h..."
3,ການ ຕັດສິນ ໃຈ ທີ່ ຈະ ປ່ຽນແປງ ກໍ ເປັນ ຂອງ ທ່ານ ...,Quyết định để thay đổi thuộc vào chính các anh...
4,ການ ເລືອກ ທີ່ ພວກ ເຈົ້າ ເຮັດ ໃນ ເວລາ ນີ້ ມີ ຄວ...,Những lựa chọn của các em ngay trước mắt là qu...
...,...,...
99995,ປະຕູສັງກະສີຈັບ-Z1961\n,cửa Kẽm xử lý-Z1961\n
99996,ກັ່ນຕອງ MS-5130\n,Lọc MS-5130\n
99997,cushion ບ່ອນນັ່ງ mold 6cm\n,Gối ghế khuôn 6cm\n
99998,Scalpers ຍິນດີຕ້ອນຮັບ\n,Chào mừng bạn\n


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
checkpoint = 'facebook/nllb-200-distilled-600M'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model, return_tensors = 'pt')
metric = evaluate.load('sacrebleu')

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
tokenizer.src_lang = 'lao_Laoo'
tokenizer.tgt_lang = 'vie_Latn'
tokenizer.set_src_lang_special_tokens('lao_Laoo')
tokenizer.set_tgt_lang_special_tokens('vie_Latn')

In [ ]:
# translator = pipeline('translation', model=model.cpu(), tokenizer = tokenizer, src_lang='lao_Laoo', tgt_lang='vie_Latn', max_length=128)

# text = 'ການບໍລິການຂອງ Google, ສາມາດໃຊ້ໄດ້ຟຣີ, ແປຄຳສັບ, ວະລີ, ແລະໜ້າເວັບໃນທັນທີລະຫວ່າງພາສາອັງກິດ ແລະຫຼາຍກວ່າ 100 ພາສາອື່ນໆ.'
# ref = 'Dịch vụ của Google, được cung cấp miễn phí, dịch nhanh các từ, cụm từ và trang web giữa tiếng Anh và hơn 100 ngôn ngữ khác.'
# pred = translator(text)[0]['translation_text'] #48 bleu

In [ ]:
# y_true = tokenizer(ref, padding=True, truncation=True, max_length=256, return_tensors="pt")
# y_pred = tokenizer(pred, padding=True, truncation=True, max_length=256, return_tensors="pt")
# y_true = tokenizer.batch_decode(y_true['input_ids'], skip_special_tokens=True)
# y_pred = tokenizer.batch_decode(y_pred['input_ids'], skip_special_tokens=True)

In [ ]:
# res = metric.compute(predictions=y_pred, references=y_true)
# print(res['score'])

# Feeding Data

In [ ]:
source = df['lao_Laoo'].values.tolist()
target = df['vie_Latn'].values.tolist()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(source, target, test_size=0.2)

In [ ]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=256, return_tensors="pt")
y_train_tokenized = tokenizer(y_train, padding=True, truncation=True, max_length=256, return_tensors="pt")
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=256, return_tensors="pt")
y_val_tokenized = tokenizer(y_val, padding=True, truncation=True, max_length=256, return_tensors="pt")

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.targets['input_ids'])

    def __getitem__(self, index):
        input_ids = self.inputs["input_ids"][index].clone().detach().squeeze()
        target_ids = self.targets["input_ids"][index].clone().detach().squeeze()
        return {"input_ids": input_ids, "labels": target_ids}

In [ ]:
trainset = Dataset(X_train_tokenized, y_train_tokenized)
testset = Dataset(X_val_tokenized, y_val_tokenized)

# Postprocessing

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {'bleu': result['score']}

In [ ]:
print(len(trainset))

80000


In [ ]:
preds, labels = trainset[0]

# Work in progress

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/ColabNotebooks/data/checkpnt",
    evaluation_strategy="epoch",
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=2,
    predict_with_generate=True,
    load_best_model_at_end=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=testset,
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics
)

In [ ]:
!rm -rf /kaggle/working/nllb_checkpoints

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

30

In [ ]:
trainer.train()

You're using a NllbTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.save_model('/content/drive/MyDrive/ColabNotebooks/data/model')

In [ ]:
!zip -r file.zip /kaggle/working/final_model